In [17]:
%load_ext autoreload
%autoreload 2
import logging
import sys
import time
from dataclasses import dataclass
from enum import Enum

from autoscript_sdb_microscope_client.structures import AdornedImage, StagePosition, ManipulatorPosition
from liftout.fibsem.sample import Sample, Lamella, AutoLiftoutState, AutoLiftoutStage



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### New Sample Data Structure
Prototyping for the new sample dataclass, and filesystem structures


In [18]:
import random
import os
from pprint import pprint

from liftout.fibsem.sample import create_experiment, load_experiment
from pathlib import Path

sample_path = "manual_experiment"
sample = create_experiment(sample_path)

for i in range(1, random.randint(6, 8)):

    lamella = Lamella(path=sample.path, number=i)
    lamella.lamella_coordinates = StagePosition(i, i, i, i, i)
    lamella.landing_coordinates = StagePosition(i, i, i, i, i)

    import random
    lamella.current_state.stage   = random.choice([stage for stage in AutoLiftoutStage]) 

    for j in range(random.randint(1, 6)):
        lamella.history.append(lamella.current_state)

    sample.positions[lamella._number] = lamella


pprint(sample)
pprint(sample.positions)

# save to file
sample.save()

# load from file
sample2 = load_experiment(sample.path)

# validation
assert sample.path == sample2.path
for i in sample.positions:
    assert sample.path[i] == sample2.path[i]
    assert sample.positions[i].lamella_coordinates == sample2.positions[i].lamella_coordinates
    assert sample.positions[i].landing_coordinates == sample2.positions[i].landing_coordinates
    assert sample.positions[i]._id == sample2.positions[i]._id
    assert sample.positions[i]._petname == sample2.positions[i]._petname
    assert sample.positions[i].current_state == sample2.positions[i].current_state
    assert sample.positions[i].history == sample2.positions[i].history
    print(f"lamella {i} passed.")



# updates to dictionarys follow reference, not sure about others

# sample.save()


{'log_path': 'c:\\users\\pcle0002\\documents\\repos\\autoliftout\\liftout\\log\\manual_experiment_2022-07-22.01-59-09PM\\logfile.log',
 'name': 'manual_experiment_2022-07-22.01-59-09PM',
 'path': 'c:\\users\\pcle0002\\documents\\repos\\autoliftout\\liftout\\log\\manual_experiment_2022-07-22.01-59-09PM',
 'positions': []}
Sample: 
        Path: c:\users\pcle0002\documents\repos\autoliftout\liftout\log\manual_experiment_2022-07-22.01-59-09PM
        State: None
        Lamella: 6
        
{1: 
        Lamella 1 (01-quiet-alpaca). 
        Lamella Coordinates: StagePosition(x=1, y=1, z=1, t=1, r=1), 
        Landing Coordinates: StagePosition(x=1, y=1, z=1, t=1, r=1), 
        Current Stage: AutoLiftoutStage.MillJCut,
        Last Completed: AutoLiftoutStage.Setup
        History: 6 stages completed (['MillJCut', 'MillJCut', 'MillJCut', 'MillJCut', 'MillJCut', 'MillJCut']).
        ,
 2: 
        Lamella 2 (02-busy-stork). 
        Lamella Coordinates: StagePosition(x=2, y=2, z=2, t=2, r=

In [35]:
import pandas as pd



df = sample_to_dataframe(sample)

display(df)



,number,petname,lamella.x,lamella.y,lamella.z,lamella.r,lamella.t,lamella.coordinate_system,landing.x,landing.y,landing.z,landing.r,landing.t,landing.coordinate_system,landing_selected,current_stage,last_completed,last_timestamp,history:
0,1,01-quiet-alpaca,1,1,1,1,1,None,1,1,1,1,1,None,False,MillJCut,AutoLiftoutStage.Setup,None,6
1,2,02-busy-stork,2,2,2,2,2,None,2,2,2,2,2,None,False,Landing,AutoLiftoutStage.Setup,None,2
2,3,03-decent-calf,3,3,3,3,3,None,3,3,3,3,3,None,False,Thinning,AutoLiftoutStage.Setup,None,2
3,4,04-choice-tarpon,4,4,4,4,4,None,4,4,4,4,4,None,False,Initialisation,AutoLiftoutStage.Setup,None,1
4,5,05-comic-marten,5,5,5,5,5,None,5,5,5,5,5,None,False,Reset,AutoLiftoutStage.Setup,None,6
5,6,06-living-insect,6,6,6,6,6,None,6,6,6,6,6,None,False,Finished,AutoLiftoutStage.Setup,None,6


### New Protocol
- Splitting the protocol into separate files. 
    - load into a single settings, dictionary
    - save protocol into individual files 
    - load protocol individually
    - possibly change to loading experiments

In [143]:
base_path = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout"

from pprint import pprint
from liftout.utils import load_config_v2

system_fname = os.path.join(base_path, "config", "system.yaml")
calibration_fname = os.path.join(base_path, "config", "calibration.yaml")
protocol_fname = os.path.join(base_path, "protocol", "protocol.yaml")

settings = load_config_v2(
    system_fname, 
    calibration_fname, 
    protocol_fname
)

pprint(settings)


{'calibration': {'calibration': {'drift_correction_hfw_highres': 8e-05,
                                 'eucentric_height_eb': 0.004,
                                 'eucentric_height_ib': 0.0165,
                                 'eucentric_height_tolerance': 0.0005,
                                 'eucentric_hfw_highres': 0.0002,
                                 'eucentric_hfw_lowres': 0.0009,
                                 'eucentric_hfw_midres': 0.0004,
                                 'max_eb_hfw': 0.0027,
                                 'max_ib_hfw': 0.0009,
                                 'max_working_distance_eb': 0.006,
                                 'stage_height_limit': 0.0037},
                 'gamma': {'correction': 1.0,
                           'max_gamma': 1.8,
                           'min_gamma': 0.15,
                           'scale_factor': 0.01,
                           'threshold': 46.0},
                 'imaging': {'autocontrast': 1.0,
          

In [145]:


def update_protocol(settings: dict, step: str, fname: Path) -> None:

    print("step: ", step)
    pprint(settings["protocol"][step])
    print("fname: ", fname)
    print("-"*50)


# from liftout.fibsem.milling import get_milling_protocol_stages
# protocol_stages = get_milling_protocol_stages(settings, "lamella")

# pprint(protocol_stages)

for k in settings["protocol"].keys():
    update_protocol(settings, k, None)


step:  lamella
{'lamella_height': 6e-06,
 'lamella_width': 1.2e-05,
 'milling_current': 7.6e-09,
 'milling_depth': 1.5e-05,
 'protocol_stages': [{'milling_current': 7.6e-09,
                      'offset': 2e-06,
                      'trench_height': 1e-05},
                     {'milling_current': 7.6e-09,
                      'offset': 0.0,
                      'trench_height': 3e-06}],
 'size_ratio': 1.0}
fname:  None
--------------------------------------------------
step:  jcut
{'extra_bit': 8e-06,
 'height': 5e-06,
 'jcut_angle': 6.0,
 'length': 1.2e-05,
 'milling_current': 2e-09,
 'milling_depth': 3e-06,
 'trench_thickness': 1e-06}
fname:  None
--------------------------------------------------
step:  weld
{'depth': 2e-07, 'height': 5e-06, 'rotation': 0.0, 'width': 3e-06}
fname:  None
--------------------------------------------------
step:  sharpen
{'bias': 2.5e-07,
 'depth': 8e-06,
 'height': 2.5e-06,
 'hfw': 0.00015,
 'milling_current': 7.6e-09,
 'needle_angle': 40.0,
 'ti

: 

: 

### Update Image Settings
new functional image settings update

In [36]:
# update_image_settings
%load_ext autoreload
%autoreload 2




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:

from liftout.fibsem.sample import load_experiment
sample_path = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\log\1245_2022-07-21.03-52PM"
sample = load_experiment(sample_path)
settings = utils.load_config_v2()


In [43]:
image_settings = update_image_settings_v3(settings, path=sample.path)

pprint(image_settings)
pprint(settings["calibration"]["imaging"])


image_settings = update_image_settings_v3(settings, path=image_settings.save_path)

pprint(image_settings)
pprint(settings["calibration"]["imaging"])




ImageSettings(resolution='1536x1024', dwell_time=1e-06, hfw=0.00015, autocontrast=1.0, beam_type=<BeamType.ELECTRON: 1>, save=False, label='2022-07-21.06-07-49PM', gamma=GammaSettings(enabled=1.0, min_gamma=0.15, max_gamma=1.8, scale_factor=0.01, threshold=46.0), save_path='c:\\users\\pcle0002\\documents\\repos\\autoliftout\\liftout\\log\\1245_2022-07-21.03-52PM\\1245_2022-07-21.03-52PM')
{'autocontrast': 1.0,
 'dwell_time': 1e-06,
 'horizontal_field_width': 0.00015,
 'imaging_current': 2e-11,
 'milling_current': 2e-09,
 'resolution': '1536x1024',
 'save': 0.0}
ImageSettings(resolution='1536x1024', dwell_time=1e-06, hfw=0.00015, autocontrast=1.0, beam_type=<BeamType.ELECTRON: 1>, save=False, label='2022-07-21.06-07-49PM', gamma=GammaSettings(enabled=1.0, min_gamma=0.15, max_gamma=1.8, scale_factor=0.01, threshold=46.0), save_path='c:\\users\\pcle0002\\documents\\repos\\autoliftout\\liftout\\log\\1245_2022-07-21.03-52PM\\1245_2022-07-21.03-52PM')
{'autocontrast': 1.0,
 'dwell_time': 1e-

Sample: 
        Path: c:\users\pcle0002\documents\repos\autoliftout\liftout\log\1245_2022-07-21.03-52PM\1245_2022-07-21.03-52PM
        State: None
        Lamella: 0
        